# Klassifikation 

In diesem Notebook trainieren wir einen Classifier. Aus Beispielen lernt der Classifier, zu welcher Klasse ein Element gehört, und kann neue Elemente mit den gelernten Regeln einordnen. 

Wir lesen zunächst Dokumente aus 4 Wikipedia-Kategorien ein.

In [1]:
import SaveWiki

SaveWiki.downloadWikiCat('Infektionskrankheit','infekt')
SaveWiki.downloadWikiCat('Krankheitsbild in der Pneumologie','pneumologie')
SaveWiki.downloadWikiCat('Bösartige Tumorbildung','krebs')
SaveWiki.downloadWikiCat('Krankheitsbild in der Kardiologie','kardiologie')

In [2]:
import codecs 

def readtext(dateiname):
    text = ''
    d = codecs.open(dateiname,'r','utf8')
    for zeile in d:
        text += str(zeile)
    d.close()

    return text

In [3]:
bsptext= readtext('krebs/Adenokarzinom.txt')
print(bsptext[:300])

Als Adenokarzinom bezeichnet man einen bösartigen (malignen) von der Deckzellschicht (Epithel) ausgehenden Tumor, der aus Drüsengewebe hervorgegangen ist. Die gutartige (benigne) Zellveränderung von Drüsengewebe nennt man dagegen Adenom.
Adenokarzinome kommen vor allem im Bereich der Verdauungsorgan


In [4]:
import nltk
import re
from HanTa import HanoverTagger as ht
from collections import Counter

stopwords = nltk.corpus.stopwords.words('german')
tagger = ht.HanoverTagger('morphmodel_ger.pgz')

def closed_class(pos):
    if pos[0] == '$':
        return True
    elif pos in ["APPR", "APPRART", "APPO", "APZR", "ART", "KOUI", "KOUS", "KON", "KOKOM", "PDS", "PDAT", "PIS", "PIAT", "PIDAT", "PPER", "PPOSS", "PPOSAT", "PRELS", "PRELAT", "PRF", "PWS", "PWAT", "PWAV", "PAV", "PTKZU", "PTKNEG", "VAFIN", "VAIMP", "VAINF", "VAPP", "VMFIN", "VMINF", "VMPP"]:
        return True
    
    return False

def features_from_text(text):
    wordcounts = Counter()
    tlen = 0
    
    satzliste =  nltk.sent_tokenize(text,language='german')
    for satz in satzliste:
        tokens =  nltk.word_tokenize(satz,language='german')
        tokens = [lemma for (word,lemma,pos) in tagger.tag_sent(tokens) if not closed_class(pos)]
        tokens = [t for t in tokens if t.lower() not in stopwords]
        tokens = [t for t in tokens if re.search('^\w+$',t)]
        tlen += len(tokens)
        wordcounts.update(tokens)

    return {w:wordcounts[w]/tlen for w in wordcounts}

Wir versuchen aus jede Klasse 50 Dokumente zu lesen, die nicht extrem kurz oder lang sind.

In [5]:
import glob

def read_data(directories):
    docs = []
    for directory in directories:
        dirsize = 0
        for file in glob.glob(directory+"/*.txt"):
            text = readtext(file)
            if len(text) > 500 and len(text) < 10000:
                docs.append((features_from_text(text),directory))
                dirsize += 1
            if dirsize >= 50:
                break
    return docs

data = read_data(['infekt','krebs','pneumologie','kardiologie'])

Wir mischen die Daten und teilen in Test- und Trainingsdaten

In [6]:
import random

random.shuffle(data)
train_data = data[:160]
test_data = data[160:]

Wir schauen uns jetzt mal ein Dokument an:

In [7]:
train_data[27]

({'Duodenalkarzinom': 0.00966183574879227,
  'bezeichnen': 0.004830917874396135,
  'bösartig': 0.004830917874396135,
  'malign': 0.004830917874396135,
  'Tumor': 0.03864734299516908,
  'Duodenums': 0.004830917874396135,
  'Zwölffingerdarm': 0.00966183574879227,
  'Risikofaktor': 0.00966183574879227,
  'gelten': 0.004830917874396135,
  'familiär': 0.004830917874396135,
  'adenomatöse': 0.004830917874396135,
  'polyposis': 0.004830917874396135,
  'obligat': 0.004830917874396135,
  'Präkanzerose': 0.004830917874396135,
  'Morbus': 0.004830917874396135,
  'Crohn': 0.004830917874396135,
  'Zöliakie': 0.004830917874396135,
  'Pathologie': 0.00966183574879227,
  'Makroskopische': 0.004830917874396135,
  'mikroskopisch': 0.004830917874396135,
  'duodenalkarzinoms': 0.00966183574879227,
  'Morphologisch': 0.004830917874396135,
  'wachsen': 0.004830917874396135,
  'meist': 0.00966183574879227,
  'polypoid': 0.004830917874396135,
  'kommen': 0.01932367149758454,
  'szirrhös': 0.004830917874396135

Wir haben jetzt für jedes Dokument einene Merkmalsvektor. Man merke: der Vektor ist eigentlich so lang wie die Anzahl der unterschiedlichen Wörter in der ganzen Sammlung. Alle Wörter, die nicht erwähnt werden haben den Wert 0. 

Wörter, die nur in ein oder zwei Dokumementen vorkommen sind, für die Klassifikation nicht besonders nützlich. 
Wir nutzen nachher nur die Wörter, die in mindestens 5 Dokumente vorkommen. Um das vorzubereiten, berechnen wir für alle Wörter die Dokumentfrequenz>

In [8]:
docfreq = Counter()
for (wfreq,c) in train_data:
    docfreq.update(wfreq.keys())

## Klassifikation mit Scikit Learn

Die Bibliothek Scikit Learn stellt verschiedene Klassifikationsmodelle zur Verfügung. Wir müssen jetzt richtige Merkmalsvektoren aufbauen, bei denen die Position die Bedeutung einer Zahl bestimmt. Dazu machen wir erst eine feste Liste mit allen Wörtern. Wörter, die zu selten sind lassen wir weg.

In [9]:
from sklearn import linear_model, datasets


allfeatures = [w for w in docfreq if docfreq[w] > 4]

def make_feat_vec(featmap,featlist):
    vec = []
    for f in featlist:
        vec.append(featmap.get(f,0.0))
    return vec

train_vec =  [make_feat_vec(feats,allfeatures) for feats,cls in train_data]
train_label = [cls for feats,cls in train_data]

Wir schauen uns mal einen Vektor an:

In [10]:
train_vec[137]

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.026143790849673203,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.006535947712418301,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.006535947712418301,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.006535947712418301,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.013071895424836602,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.013071895424836602,
 0.006535947712418301,
 0.0,
 0.0,
 0.0,
 0.006535947712418301,
 0.0196078431372549,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.006535947712418301,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.006535947712418301,
 0.006535947712418301,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.006535947712418301,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.006535947712418301,
 0.0,
 0.0,
 0.006535947712418301,
 0.

In [11]:
train_label[137]

'krebs'

Ein einfaches, aber effektives Klassifikationsmodell, das meistens gute Ergebnisse liefert, ist die logistische Regression.

In [12]:
logreg = linear_model.LogisticRegression(C=1e9,verbose=True)
logreg.fit(train_vec,train_label)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s finished


LogisticRegression(C=1000000000.0, verbose=True)

Wir können jetzt mit dem trainierten Classifier ein neues Dokument klassifizieren:

In [13]:
v = make_feat_vec(test_data[17][0],allfeatures) 
logreg.predict([v])

array(['kardiologie'], dtype='<U11')

Richtig ist:

In [14]:
test_data[17][1]

'kardiologie'

Oder direkt für alles Testdaten:

In [15]:
test_vec = [make_feat_vec(feats,allfeatures) for feats,cls in test_data]
test_label = [cls for feats,cls in test_data]

pred_label = list(logreg.predict(test_vec))

In [16]:
correct = 0
for i in range(len(test_label)):
    if test_label[i] == pred_label[i]:
        correct+=1
print("{0:.1f} Prozent korrekt".format(100* float(correct)/len(test_label)))

85.0 Prozent korrekt


Wo sind aber die Fehler aufgetreten? Un diese Frage zu beantworten, ist sogenannte eine _Konfusionsmatrix_ sehr hilfreich. NLTK hat eine Funktion zum erstellen eines Konfusionsmatrizen:

In [64]:
cm = nltk.ConfusionMatrix(test_label, pred_label)
print(cm)

            |     k     p |
            |     a     n |
            |     r     e |
            |     d     u |
            |     i     m |
            |  i  o     o |
            |  n  l  k  l |
            |  f  o  r  o |
            |  e  g  e  g |
            |  k  i  b  i |
            |  t  e  s  e |
------------+-------------+
     infekt | <8> .  .  2 |
kardiologie |  1<12> .  . |
      krebs |  .  .<10> . |
pneumologie |  .  1  . <6>|
------------+-------------+
(row = reference; col = test)



Es gibt verschiedene andere Klassifikatorn, die wie nutzen können:

In [18]:
from sklearn import neighbors
knn = neighbors.KNeighborsClassifier(n_neighbors = 3)
knn.fit(train_vec,train_label)
pred_label = list(knn.predict(test_vec))
correct = 0
for i in range(len(test_label)):
    if test_label[i] == pred_label[i]:
        correct+=1
print("{0:.1f} Prozent korrekt".format(100* float(correct)/len(test_label)))

55.0 Prozent korrekt


In [19]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier()
rf.fit(train_vec,train_label)
pred_label = list(rf.predict(test_vec))
correct = 0
for i in range(len(test_label)):
    if test_label[i] == pred_label[i]:
        correct+=1
print("{0:.1f} Prozent korrekt".format(100* float(correct)/len(test_label)))

62.5 Prozent korrekt


C:\Users\wartena\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
